In [135]:
import xxhash

Set the modulus we will use for our hash function (the size of the table)

In [17]:
n = 13

In [18]:
keys = ["welcome", "to", "the", "jungle"]

In [19]:
positions = [ xxhash.xxh3_64_intdigest(k) % n for k in keys ]

In [20]:
positions

[7, 9, 10, 0]

In [21]:
keys = ["welcome", "to", "the", "jungle", "we", "got", "fun", "and", "games"]

In [22]:
positions = [ xxhash.xxh3_64_intdigest(k) % n for k in keys ]

In [23]:
positions

[7, 9, 10, 0, 2, 7, 7, 0, 11]

Let's try with the full lyrics

In [24]:
lyrics = """
Welcome to the jungle
We got fun and games
We got everything you want
Honey, we know the names
We are the people that can find
Whatever you may need
If you got no money, honey
We got your disease
In the jungle, welcome to the jungle
Watch it bring you to your knees, knees
Ooh-ah, I wanna watch you bleed
Welcome to the jungle
We take it day by day
If you want it, you're gonna bleed
But that's the price you pay
You're a very sexy girl
Who's very hard to please
You can taste the bright lights
But you won't get them for free
In the jungle, welcome to the jungle
Feel my, my, my, my serpentine
I, I wanna hear you scream
Welcome to the jungle
It's worse here everyday
You learn to live like an animal
In the jungle where we play
You got a hunger for what you see
You'll take it eventually
You can have anything you want
But you better not take it from me
In the jungle, welcome to the jungle
Watch it bring you to your knees, knees
Ooh-ah, I'm gonna watch you bleed
And when you're high, you never
Ever want to come down
So down, so down, so down, yeah
Aw
You know where you are?
You're in the jungle, baby
You're gonna die
In the jungle, welcome to the jungle
Watch it bring you to your knees, knees
In the jungle, welcome to the jungle
Feel my, my, my, my serpentine
In the jungle, welcome to the jungle
Watch it bring you to your knees, knees
In the jungle, welcome to the jungle
Watch it bring to your...
It's gonna bring you down, huh
"""

Get the set of all words in the lyrics

In [39]:
words = [ x.strip('\"') for l in lyrics.split('\n') for x in l.rstrip().split(' ')]

In [40]:
len(set(words))

119

We want a hash table large enough to accomodate all the words, we choose a prime > 119

In [50]:
n = 197

In [51]:
hashes = [xxhash.xxh3_64_intdigest(w) for w in words]

In [52]:
len(set(hashes))

119

In [53]:
hashes_modn = [h % n for h in hashes]

In [55]:
len(set(hashes_modn))

94

So we can already see that there are no collision in 64-bit hash space, but some collisions when we take the hash % n.

### Let's try making a simple separate chaining hash table

In [88]:
class SimpleHash:
    def __init__(self, size):
        self.size = size
        self.arr = [ [] for _ in range(self.size)]
        
    def add(self, k):
        i = xxhash.xxh3_64_intdigest(k) % self.size
        self.arr[i].append(k)
    
    def exists(self, k):
        i = xxhash.xxh3_64_intdigest(k) % self.size
        try:
            idx = self.arr[i].index(k)
            return True
        except ValueError as e:
            return False

    def __str__(self):
        return f"size: {self.size}\narr: {self.arr}"

In [103]:
shash = SimpleHash(13)

In [115]:
keys = ["welcome", "to", "the", "jungle"]

In [116]:
for k in keys:
    shash.add(k)

In [117]:
print(shash)

size: 13
arr: [['jungle', 'jungle'], [], [], [], [], [], [], ['welcome', 'welcome'], [], ['to', 'to'], ['the', 'the'], [], []]


In [118]:
for k in keys:
    print(f"{k} exists? {shash.exists(k)}")

welcome exists? True
to exists? True
the exists? True
jungle exists? True


In [119]:
shash.exists("w00t")

False

In [120]:
keys = ["welcome", "to", "the", "jungle", "we", "got", "fun", "and", "games"]

In [121]:
shash_with_coll = SimpleHash(13)

In [122]:
for k in keys:
    shash_with_coll.add(k)

In [123]:
print(shash_with_coll)

size: 13
arr: [['jungle', 'and'], [], ['we'], [], [], [], [], ['welcome', 'got', 'fun'], [], ['to'], ['the'], ['games'], []]


In [124]:
for k in keys:
    print(f"{k} exists? {shash_with_coll.exists(k)}")

welcome exists? True
to exists? True
the exists? True
jungle exists? True
we exists? True
got exists? True
fun exists? True
and exists? True
games exists? True


In [125]:
shash_with_coll.exists("w00t")

False

### Of course, in Python, we can just use the built-in `dict`; it's a hash table!

In [126]:
hash_table = {}

In [128]:
for k in keys:
    hash_table[k] = True

In [129]:
print(hash_table)

{'welcome': True, 'to': True, 'the': True, 'jungle': True, 'we': True, 'got': True, 'fun': True, 'and': True, 'games': True}


In [130]:
for k in keys:
    print(f"{k} exists? {k in hash_table}")

welcome exists? True
to exists? True
the exists? True
jungle exists? True
we exists? True
got exists? True
fun exists? True
and exists? True
games exists? True


In [131]:
"w00t" in hash_table

False

### Finally we can, of course, associate keys with *values*

In [132]:
hash_table = {}
for i,k in enumerate(keys):
    hash_table[k] = i

In [133]:
print(hash_table)

{'welcome': 0, 'to': 1, 'the': 2, 'jungle': 3, 'we': 4, 'got': 5, 'fun': 6, 'and': 7, 'games': 8}


In [134]:
for k in keys:
    print(f"position of {k} in original list is {hash_table[k]}")

position of welcome in original list is 0
position of to in original list is 1
position of the in original list is 2
position of jungle in original list is 3
position of we in original list is 4
position of got in original list is 5
position of fun in original list is 6
position of and in original list is 7
position of games in original list is 8


How might this be useful for us?  Thinking ahead — what if the keys are short "seeds" from the genome (e.g. k-mers) and values are the positions of these seeds ... 